In [1]:
import cv2
import json
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon
from collections import defaultdict
from typing import List, Dict
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from collections import deque
from utils import *

from sklearn.neighbors import KDTree
from skimage.measure import EllipseModel

In [2]:
# Placeholder: Already implemented externally
def load_coco_inferences(coco_json_path: str) -> Dict[int, List[Dict]]:
    """
    Load inferenced COCO file and return a mapping from image_id to list of detections.

    Args:
        coco_json_path (str): Path to the COCO-format JSON file.

    Returns:
        Dict[int, List[Dict]]: Mapping from image_id to list of detection dicts.
    """
    import json

    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Build mapping from image_id to image file name (if needed)
    image_id_to_filename = {img['id']: img['file_name'] for img in coco_data.get('images', [])}

    # Group annotations by image_id
    image_id_to_anns = defaultdict(list)
    for ann in coco_data.get('annotations', []):
        image_id_to_anns[ann['image_id']].append(ann)

    return dict(image_id_to_anns)


def project_ellipse_center_to_world(camera_meta: np.ndarray, ellipse_center_px: np.ndarray) -> Dict:
    """
    Project ellipse center in image to a ray in world coordinates.
    Returns {'origin': np.ndarray, 'direction': np.ndarray}
    """
    
    film_coords = spherical_unprojection(ellipse_center_px[0], ellipse_center_px[1], 1, camera_meta['width'], camera_meta['height'])
    x_local, y_local, z_local = transform_to_local_crs(film_coords[0], film_coords[1], film_coords[2], camera_meta)
    ray_vertex = np.array([x_local, y_local, z_local])

    cam_center = np.array([camera_meta['x'], camera_meta['y'], camera_meta['z']])
    ray_ori = ray_vertex - cam_center
    ray_data = {
        'origin': cam_center,
        'direction': ray_ori
    }
    return ray_data

def spatial_temporal_group_sort(
    gdf_grouped,
    time_column="gps_sec_s_",
    x_col="x_m_",
    y_col="y_m_",
    z_col="z_m_",
    radius=10.0,
    output_column="sort_index"
):
    """
    Two-stage spatial-temporal sort on groupby(image_id) object.
    
    Parameters:
        gdf_grouped: GroupBy object, grouped by 'image_id'.
        time_column (str): Column for GPS time (shared within each group).
        x_col, y_col, z_col (str): Position columns (shared within each group).
        radius (float): Search radius in meters.
        output_column (str): Name of output sort index column.
    
    Returns:
        GeoDataFrame: With new sort_index column.
        GroupBy: Sorted groupby object (ordered by sort_index).
    """

    # Step 1: Extract one row per group to represent its position and time
    group_meta = (
        gdf_grouped[[time_column, x_col, y_col, z_col]]
        .first()
        .reset_index()
    )

    # Sort groups by time (Stage 1)
    group_meta = group_meta.sort_values(by=time_column).reset_index(drop=True)
    positions = group_meta[[x_col, y_col, z_col]].values
    image_ids = group_meta["image_id"].values

    tree = KDTree(positions)
    used = np.zeros(len(group_meta), dtype=bool)
    final_order_indices = []

    prev_anchor_idx = None

    for current_anchor_idx in range(len(group_meta)):
        if used[current_anchor_idx]:
            continue

        current_pos = positions[current_anchor_idx]

        if prev_anchor_idx is None:
            group_indices = [current_anchor_idx]
        else:
            prev_pos = positions[prev_anchor_idx]
            direction = current_pos - prev_pos
            direction_norm = np.linalg.norm(direction)
            if direction_norm < 1e-6:
                direction = np.array([1, 0, 0])
            else:
                direction = direction / direction_norm

            neighbor_indices = tree.query_radius([current_pos], r=radius)[0]
            neighbor_indices = [i for i in neighbor_indices if not used[i]]

            if not neighbor_indices:
                continue

            neighbor_positions = positions[neighbor_indices]
            projections = neighbor_positions @ direction
            sorted_local = np.array(neighbor_indices)[np.argsort(projections)]
            group_indices = sorted_local.tolist()

        used[group_indices] = True
        final_order_indices.extend(group_indices)
        prev_anchor_idx = current_anchor_idx

    # Create mapping from image_id to sort index
    sort_index_map = {image_ids[idx]: i for i, idx in enumerate(final_order_indices)}

    # Apply mapping to original GeoDataFrame
    gdf_with_sort = gdf_grouped.obj.copy()
    gdf_with_sort[output_column] = gdf_with_sort["image_id"].map(sort_index_map)

    # Return new dataframe and sorted groupby object
    gdf_sorted = gdf_with_sort.sort_values(by=output_column)
    return gdf_sorted.groupby("image_id", sort=False)


In [3]:
def fit_ellipse_from_mask(mask: np.ndarray):
    """Fit an ellipse to a binary mask."""
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt = max(contours, key=cv2.contourArea)
    if len(cnt) < 5:
        return None
    ellipse = cv2.fitEllipse(cnt)
    return ellipse  # (center(x, y), (major, minor), angle)

class Ray():
    def __init__(self, origin: np.ndarray, direction: np.ndarray, frame_id: int, candidate_id=None):
        self.origin = origin
        self.direction = direction / np.linalg.norm(direction)
        self.frame_id = frame_id
        self.candidate_id = candidate_id  # which candidate this ray is currently associated with

# Intersection class definition
class Intersection:
    def __init__(self, point, ray_pair, dist, length):
        self.point = point          # np.array([x, y, z])
        self.ray_pair = ray_pair    # tuple of (prev_idx, new_idx)
        self.dist = dist            # float, distance between rays at intersection
        self.length = length        # float, sum of distances from ray origins to intersection

def compute_ray_intersection(ray1: Ray, ray2: Ray, radius: float = 10.0, distance_threshold: float = 0.5) -> np.ndarray:
    """
    Return intersection point if rays intersect in their forward direction and are close enough.
    Otherwise, return None.
    """
    p1, d1 = ray1.origin, ray1.direction
    p2, d2 = ray2.origin, ray2.direction

    v12 = p2 - p1
    d1_dot_d2 = np.dot(d1, d2)
    denom = 1 - d1_dot_d2 ** 2
    if abs(denom) < 1e-6:
        return None, None, None  # Nearly parallel, no intersection in forward direction
    t1 = (np.dot(v12, d1) - np.dot(v12, d2) * d1_dot_d2) / denom
    t2 = - (np.dot(v12, d2) - np.dot(v12, d1) * d1_dot_d2) / denom
    if t1 < 0 or t2 < 0 or t1 > radius or t2 > radius:
        return None, None, None # Closest approach is behind at least one ray's origin
    point1 = p1 + t1 * d1
    point2 = p2 + t2 * d2
    dist = np.linalg.norm(point1 - point2)

    if dist > distance_threshold:
        return None, None, None  # Closest points are too far apart to be considered an intersection
    
    intersect = (point1 + point2) / 2
    if intersect[2] >= min(p1[2], p2[2]) - 2.0 or intersect[2] <= min(p1[2], p2[2]) - 3.8:
        return None, None, None
    return intersect, dist, max(t1, t2)

def cluster_intersections(intersections: List[np.ndarray], distance_threshold: float = 0.5) -> List[List[int]]:
    """Cluster intersection points by spatial proximity using DBSCAN."""
    if len(intersections) == 0:
        return []
    X = np.stack(intersections)
    db = DBSCAN(eps=distance_threshold, min_samples=1).fit(X)
    labels = db.labels_
    clusters = []
    for label in set(labels):
        cluster = np.where(labels == label)[0].tolist()
        clusters.append(cluster)
    return clusters

def estimate_disc_geometry(intersection_points: List[np.ndarray]) -> Dict:
    """Initialize manhole model: center, upward normal, and radius."""
    center = np.mean(intersection_points, axis=0)
    normal = np.array([0, 0, 1])
    radius = 0.3  # fixed 30cm
    return {"center": center, "normal": normal, "radius": radius}

def detect_and_localize_manhole(gdf: gpd.GeoDataFrame, verbose=True) -> gpd.GeoDataFrame:
    """
    Detect and localize manholes from georeferenced mobile mapping imagery.
    Uses a list for rays, updates candidates incrementally, and manages candidate/ray lifecycle.
    """
    return 


# Trajectory & COCO

## Pred

In [4]:
# Paths to the three COCO files
coco_paths = [
    '/mnt/Data/StreetView/data/neuchatel/trn_COCO_panoptic_detections.json',
    '/mnt/Data/StreetView/data/neuchatel/val_COCO_panoptic_detections.json',
    '/mnt/Data/StreetView/data/neuchatel/tst_COCO_panoptic_detections.json'
]

merged_file_path = '/mnt/Data/StreetView/data/neuchatel/COCO_panoptic_detections.json'
# Load all three COCO files
coco_datas = []
for path in coco_paths:
    with open(path, 'r') as f:
        coco_datas.append(json.load(f))

# Merge 'images' and remove duplicates by 'file_name'
all_images = []
seen_file_names = set()
for coco in coco_datas:
    for img in coco['images']:
        fname = img['file_name']
        if fname not in seen_file_names:
            all_images.append(img)
            seen_file_names.add(fname)

# Merge 'annotations'
all_annotations = []
for coco in coco_datas:
    all_annotations.extend(coco.get('annotations', []))

# Use categories from the first file (assuming all are the same)
categories = coco_datas[0].get('categories', [])

# Build merged COCO data
merged_coco = {
    'images': all_images,
    'annotations': all_annotations,
    'categories': categories
}

# Remove parent folders from file_name in coco_data['images'] and rename 'image_id' to 'id'
for img in merged_coco['images']:
    img['file_name'] = img['file_name'].split('/')[-1]
    if 'image_id' in img:
        img['id'] = img.pop('image_id')

merged_coco['categories'] = [{
    'id': int(0), 
    'name': 'manhole', 
    'supercategory': 'none'}]
# Save the modified coco_data back to coco_file_path
with open(merged_file_path, 'w') as f:
    json.dump(merged_coco, f)

In [5]:
# Read the trajectory CSV as a DataFrame
traject = pd.read_csv('/mnt/Data/StreetView/data/neuchatel/ne_traject.csv')
coco_file_path = '/mnt/Data/StreetView/data/neuchatel/COCO_panoptic_detections.json'


# Construct geometry from x_m_, y_m_, z_m_
traject['geometry'] = traject.apply(lambda row: Point(row['x_m_'], row['y_m_'], row['z_m_']), axis=1)
gdf = gpd.GeoDataFrame(traject, geometry='geometry', crs="epsg:2056")
gdf = gpd.GeoDataFrame(traject)

# Load COCO data
with open(coco_file_path, 'r') as f:
    coco_data = json.load(f)


In [6]:
# --- Match gdf with coco_data by file_name ---
# Build a DataFrame from coco_data['images']
coco_images_df = pd.DataFrame(coco_data['images'])
coco_images_df.id = coco_images_df.id.astype(int)

# Remove '.jpg' from coco file_name for matching
coco_images_df['file_name_no_ext'] = coco_images_df['file_name'].str.replace('.jpg', '', regex=False)

# If gdf has a column with file name (without extension), match on that
# Let's assume the column is 'file_name' in gdf (without .jpg)
# If not, adjust accordingly
if 'file_name' not in gdf.columns:
    raise ValueError("gdf must have a 'file_name' column to match with COCO images.")

# Merge gdf with coco_images_df to get the COCO image id
gdf = gdf.merge(
    coco_images_df[['id', 'file_name_no_ext', 'width', 'height']],
    left_on='file_name',
    right_on='file_name_no_ext',
    how='right'
)
gdf = gdf.rename(columns={'id': 'coco_image_id'})
gdf.drop(columns=['file_name_no_ext'], inplace=True)

# --- COCO annotation processing as before ---
# Prepare a list of annotation records
records = []
for ann in coco_data['annotations']:
    if float(ann['score']) < 0.9 or float(ann['area']) > 10000 or float(ann['area']) < 400:
        continue
    image_id = ann['image_id']
    segmentation = ann['segmentation'][0]

    coords = [(segmentation[i], segmentation[i+1]) for i in range(0, len(segmentation), 2)]
    geometry = Polygon(coords)
    records.append({
        'image_id': image_id,
        'category_id': ann.get('category_id', None),
        'geometry': geometry,
        'annotation_id': ann.get('id', None),
        'score': ann.get('score', None),
        'area': ann.get('area', None)
    })

# Create a GeoDataFrame
ann_gdf = gpd.GeoDataFrame(records, geometry='geometry', crs="epsg:2056")

# Merge gdf with ann_gdf to get prediction mask
gdf = gdf.merge(
    ann_gdf,
    left_on='coco_image_id',
    right_on='image_id',
    how='right'
)
gdf.drop(columns=['coco_image_id'], inplace=True)

grouped = spatial_temporal_group_sort(gdf.groupby('image_id'))

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 4))
# ann_gdf['score'].hist(bins=50)
# plt.title('Distribution of annotation scores')
# plt.xlabel('Score')
# plt.ylabel('Count')
# plt.show()

In [ ]:
# plt.figure(figsize=(8, 4))
# ann_gdf['area'][ann_gdf['area']<1000].hist(bins=50)
# plt.title('Distribution of annotation scores')
# plt.xlabel('Score')
# plt.ylabel('Count')
# plt.show()

In [7]:
# Parameters
intersection_distance_threshold = 0.5  # meters, for intersection and clustering
candidate_update_threshold = 1.0       # meters, for candidate consistency
candidate_missing_limit = 5            # number of consecutive frames without new rays before removal
radius = 20.0

# Candidate class for storing candidate attributes
class Candidate:
    def __init__(self, center, intersections, mean_intersection_length, mean_intersection_dist, last_seen, missing):
        self.center = center
        self.intersections = set(intersections)
        self.mean_intersection_length = mean_intersection_length
        self.mean_intersection_dist = mean_intersection_dist
        self.last_seen = last_seen
        self.missing = missing

    def update(self, center, intersections, mean_intersection_length, mean_intersection_dist, last_seen):
        self.center = center
        self.intersections = set(intersections)
        self.mean_intersection_length = mean_intersection_length
        self.mean_intersection_dist = mean_intersection_dist
        self.last_seen = last_seen
        self.missing = 0

    def increment_missing(self):
        self.missing += 1

    def to_row(self):
        center = self.center
        return {
            'elevation': center[2],
            'radius': 0.3,
            'normal_x': 0,
            'normal_y': 0,
            'normal_z': 1,
            'intersections': set(self.intersections),
            'mean_intersection_length': self.mean_intersection_length,
            'mean_intersection_dist': self.mean_intersection_dist,
            'geometry': Point(center[0], center[1])
        }

# State
ray_list = []  # List of Ray objects (never removed, indices are constant)
candidate_list = []  # List of Candidate objects
intersection_objs = []  # List of Intersection objects

iterator = tqdm(grouped, total=len(grouped), desc="Processing frames")

# Maintain sets of active indices for rays, candidates, and intersections
active_ray_indices = set()
active_candidate_indices = set()
active_intersection_indices = set()

for image_id, df_frame in iterator:
    frame_id = image_id

    camera_meta = {
        'x': df_frame.iloc[0]['x_m_'],
        'y': df_frame.iloc[0]['y_m_'],
        'z': df_frame.iloc[0]['z_m_'],
        'yaw': df_frame.iloc[0]['gpsimgdirection'] + 0.5,
        'pitch': df_frame.iloc[0]['gpspitch'],
        'roll': df_frame.iloc[0]['gpsroll'] - 0.3,
        'width': df_frame.iloc[0]['width'],
        'height': df_frame.iloc[0]['height']
    }

    # Get predicted masks for this image 
    new_rays = []
    for _, det in df_frame.iterrows():
        # Fit a minimum area ellipse to the contour of det.geometry_y and use its center as center_px
        # Get the exterior coordinates of the geometry as (N, 2) array
        coords = np.array(det.geometry_y.exterior.coords)
        # Fit ellipse to the contour
        ellipse = EllipseModel()
        success = ellipse.estimate(coords)
        if success:
            xc, yc, a, b, theta = ellipse.params
            center_px = (xc, yc)
        else:
            # Fallback to centroid if ellipse fit fails
            center_px = np.array(det.geometry_y.centroid.coords)[0]
            print('ellipse fit fails')
        ray_data = project_ellipse_center_to_world(camera_meta, center_px)
        ray = Ray(ray_data['origin'], ray_data['direction'], frame_id)
        new_rays.append(ray)

    # Add new rays to the ray list
    ray_start_idx = len(ray_list)
    ray_list.extend(new_rays)
    new_ray_indices = np.arange(ray_start_idx, len(ray_list))

    # When new rays are added, mark their indices as active
    active_ray_indices.update(new_ray_indices)

    # Compute intersections only between new rays and previous active rays, but store all intersections globally
    n_prev_rays = ray_start_idx
    n_new_rays = len(new_rays)
    new_intersection_indices = []
    if n_prev_rays > 0 and n_new_rays > 0:
        # For each pair of frame_id, only keep the intersection with the lowest dist
        # Use a dict to store: key = (frame_id_prev, frame_id_new), value = (dist, Intersection)
        for new_idx, ray_new in zip(new_ray_indices, new_rays):
            best_intersections = dict()
            for prev_idx in active_ray_indices:
                ray_prev = ray_list[prev_idx]
                # Only consider pairs from different frames
                if ray_prev.frame_id != ray_new.frame_id:
                    pt, dist, length = compute_ray_intersection(ray_prev, ray_new, radius=radius, distance_threshold=intersection_distance_threshold)
                    if pt is not None:
                        # Sort frame_id order to avoid duplicates (frame_id_a, frame_id_b)
                        frame_pair = tuple(sorted([ray_prev.frame_id, ray_new.frame_id]))
                        if frame_pair not in best_intersections or dist < best_intersections[frame_pair].dist:
                            best_intersections[frame_pair] = Intersection(
                                point=pt,
                                ray_pair=(prev_idx, new_idx),
                                dist=dist,
                                length=length
                            )
            # After collecting, add only the best intersections for each frame pair
            for intersection in best_intersections.values():
                intersection_objs.append(intersection)
                new_intersection_indices.append(len(intersection_objs) - 1)
    # Add new intersections to active set
    active_intersection_indices.update(new_intersection_indices)

    # Cluster only active intersections
    if len(active_intersection_indices) > 0:
        intersection_points = [intersection_objs[i].point for i in active_intersection_indices]
        clusters = cluster_intersections(intersection_points, distance_threshold=0.5)
        # clusters: list of lists of indices into intersection_points (not global intersection_objs)
        # Map cluster indices back to global intersection_objs indices
        clusters_global = []
        active_intersection_indices_list = list(active_intersection_indices)
        for cluster in clusters:
            clusters_global.append([active_intersection_indices_list[i] for i in cluster])
    else:
        clusters_global = []

    # For each cluster, use the 3 intersects with least length to calculate candidate
    new_candidates = []
    if len(clusters_global) > 0:
        for cluster in clusters_global:
            # cluster is a list of indices into intersection_objs (global indices)
            cand_inters = [intersection_objs[i] for i in cluster]
            # Sort by length
            cand_inters_with_idx = sorted(zip(cluster, cand_inters), key=lambda x: x[1].length)
            # Take up to 3 intersections (if less than 3, take all)
            top_k = min(3, len(cand_inters_with_idx))
            selected_inters = cand_inters_with_idx[:top_k]
            selected_points = np.array([inter[1].point for inter in selected_inters])
            selected_intersection_indices = set([inter[0] for inter in selected_inters])
            selected_lengths = [inter[1].length for inter in selected_inters]
            selected_dists = [inter[1].dist for inter in selected_inters]
            mean_intersection_length = float(np.mean(selected_lengths)) if selected_lengths else float('nan')
            mean_intersection_dist = float(np.mean(selected_dists)) if selected_dists else float('nan')
            # Find all ray indices involved in these intersections
            cluster_ray_indices = set()
            for inter in selected_inters:
                cluster_ray_indices.update(inter[1].ray_pair)
            # Compute center as mean of the selected points
            center = np.mean(selected_points, axis=0)
            new_candidates.append(
                Candidate(
                    center=center,
                    intersections=selected_intersection_indices,
                    mean_intersection_length=mean_intersection_length,
                    mean_intersection_dist=mean_intersection_dist,
                    last_seen=frame_id,
                    missing=0
                )
            )

    # Update existing candidates or add new ones (dynamic pool)
    updated_candidate_ids = set()
    # Only consider active candidates for matching
    active_candidate_indices_list = list(active_candidate_indices)
    for nc in new_candidates:
        nc_center = nc.center
        nc_inters = nc.intersections
        nc_mean_length = nc.mean_intersection_length
        nc_mean_dist = nc.mean_intersection_dist
        matched = False
        if len(active_candidate_indices) > 0:
            candidate_centers = np.array([candidate_list[cid].center for cid in active_candidate_indices_list])
            dists = np.linalg.norm(candidate_centers - nc_center, axis=1)
            within_radius_indices = np.where(dists < radius)[0]
            candidates_to_compare = [(active_candidate_indices_list[cid], candidate_list[active_candidate_indices_list[cid]]) for cid in within_radius_indices]
        else:
            candidates_to_compare = []
        for cid, cand in candidates_to_compare:
            dist = np.linalg.norm(nc_center - cand.center)
            if dist != 0 and dist < candidate_update_threshold:
                combined_inters = cand.intersections.union(nc_inters)
                relevant_inters = [
                    (i, intersection_objs[i])
                    for i in combined_inters
                    if i in active_intersection_indices
                ]
                if len(relevant_inters) > 0:
                    # Sort by length and take up to 3
                    relevant_inters_sorted = sorted(relevant_inters, key=lambda x: x[1].length)
                    top_k = min(3, len(relevant_inters_sorted))
                    selected_points = np.array([relevant_inters_sorted[i][1].point for i in range(top_k)])
                    selected_indices = set([relevant_inters_sorted[i][0] for i in range(top_k)])
                    selected_lengths = [relevant_inters_sorted[i][1].length for i in range(top_k)]
                    selected_dists = [relevant_inters_sorted[i][1].dist for i in range(top_k)]
                    new_center = np.mean(selected_points, axis=0)
                    mean_intersection_length = float(np.mean(selected_lengths)) if selected_lengths else float('nan')
                    mean_intersection_dist = float(np.mean(selected_dists)) if selected_dists else float('nan')
                    cand.update(
                        center=new_center,
                        intersections=selected_indices,
                        mean_intersection_length=mean_intersection_length,
                        mean_intersection_dist=mean_intersection_dist,
                        last_seen=frame_id
                    )
                else:
                    cand.intersections = combined_inters
                    lengths = [intersection_objs[i].length for i in combined_inters if i < len(intersection_objs)]
                    dists_ = [intersection_objs[i].dist for i in combined_inters if i < len(intersection_objs)]
                    cand.mean_intersection_length = float(np.mean(lengths)) if lengths else float('nan')
                    cand.mean_intersection_dist = float(np.mean(dists_)) if dists_ else float('nan')
                    cand.last_seen = frame_id
                    cand.missing = 0
                updated_candidate_ids.add(cid)
                matched = True
                break
        if not matched:
            candidate_list.append(
                Candidate(
                    center=nc_center,
                    intersections=set(nc_inters),
                    mean_intersection_length=nc_mean_length,
                    mean_intersection_dist=nc_mean_dist,
                    last_seen=frame_id,
                    missing=0
                )
            )
            new_cid = len(candidate_list) - 1
            updated_candidate_ids.add(new_cid)
            active_candidate_indices.add(new_cid)

    # For candidates not updated, increment missing count
    for cid in list(active_candidate_indices):
        if cid not in updated_candidate_ids:
            cand = candidate_list[cid]
            cand.increment_missing()
            if cand.missing > candidate_missing_limit:
                rays_to_remove = set()
                for iid in cand.intersections:
                    inter = intersection_objs[iid]
                    rays_to_remove.update(inter.ray_pair)
                active_ray_indices.difference_update(rays_to_remove)
                active_candidate_indices.discard(cid)
                # Also deactivate intersections that only involve rays from this candidate
                intersections_to_remove = set()
                for iid in list(active_intersection_indices):
                    inter = intersection_objs[iid]
                    if inter.ray_pair[0] in rays_to_remove or inter.ray_pair[1] in rays_to_remove:
                        intersections_to_remove.add(iid)
                active_intersection_indices.difference_update(intersections_to_remove)

# Prepare candidate centers for clustering
candidate_centers = np.array([cand.center for cand in candidate_list])
if len(candidate_centers) == 0:
    pass
else:
    # Cluster with DBSCAN using intersection_distance_threshold as eps
    db = DBSCAN(eps=intersection_distance_threshold, min_samples=1)
    labels = db.fit_predict(candidate_centers)

    # Group candidate indices by cluster label
    from collections import defaultdict
    clusters = defaultdict(list)
    for idx, label in enumerate(labels):
        clusters[label].append(idx)

    new_candidate_list = []
    old_to_new_cid = {}

    for label, indices in tqdm(clusters.items()):
        if len(indices) == 1:
            # Keep singleton candidate as is
            orig_cid = indices[0]
            new_cid = len(new_candidate_list)
            new_candidate_list.append(candidate_list[orig_cid])
            old_to_new_cid[orig_cid] = new_cid
        else:
            # Merge candidates in this cluster
            merged_intersections = set()
            merged_last_seen = -1
            merged_missing = float('inf')
            for cid in indices:
                cand = candidate_list[cid]
                merged_intersections.update(cand.intersections)
                merged_last_seen = max(merged_last_seen, cand.last_seen)
                merged_missing = min(merged_missing, cand.missing)
            intersection_points = []
            intersection_lengths = []
            intersection_dists = []
            for iid in merged_intersections:
                intersection_points.append(intersection_objs[iid].point)
                intersection_lengths.append(intersection_objs[iid].length)
                intersection_dists.append(intersection_objs[iid].dist)
            if intersection_points:
                # Use up to 3 shortest-length intersections for center and mean
                merged_inters_with_length = [(iid, intersection_objs[iid]) for iid in merged_intersections]
                merged_inters_with_length_sorted = sorted(merged_inters_with_length, key=lambda x: x[1].length)
                top_k = min(3, len(merged_inters_with_length_sorted))
                selected_points = np.array([merged_inters_with_length_sorted[i][1].point for i in range(top_k)])
                selected_indices = set([merged_inters_with_length_sorted[i][0] for i in range(top_k)])
                selected_lengths = [merged_inters_with_length_sorted[i][1].length for i in range(top_k)]
                selected_dists = [merged_inters_with_length_sorted[i][1].dist for i in range(top_k)]
                new_center = np.mean(selected_points, axis=0)
                mean_intersection_length = float(np.mean(selected_lengths)) if selected_lengths else float('nan')
                mean_intersection_dist = float(np.mean(selected_dists)) if selected_dists else float('nan')
                new_intersections = selected_indices
            else:
                centers = [candidate_list[k].center for k in indices]
                new_center = np.mean(np.array(centers), axis=0)
                mean_intersection_length = float('nan')
                mean_intersection_dist = float('nan')
                new_intersections = set()
            new_cand = Candidate(
                center=new_center,
                intersections=new_intersections if new_intersections else merged_intersections,
                mean_intersection_length=mean_intersection_length,
                mean_intersection_dist=mean_intersection_dist,
                last_seen=merged_last_seen,
                missing=merged_missing
            )
            new_cid = len(new_candidate_list)
            for k in indices:
                old_to_new_cid[k] = new_cid
            new_candidate_list.append(new_cand)

    # Replace candidate_list with merged version
    candidate_list = new_candidate_list

    # Update active_candidate_indices to new indices
    active_candidate_indices = set(old_to_new_cid[cid] for cid in active_candidate_indices if cid in old_to_new_cid)

# Output remaining active candidates as GeoDataFrame
rows = []
for cid, cand in enumerate(candidate_list):
    row = cand.to_row()
    rows.append(row)

out_gdf = gpd.GeoDataFrame(rows, geometry='geometry', crs="epsg:2056")
out_gdf.to_file(f"triangulation_points_pred_{str(int(radius))+'m'}.gpkg", driver="GPKG")



Processing frames:   0%|          | 0/4459 [00:00<?, ?it/s]

100%|██████████| 1855/1855 [00:00<00:00, 22845.73it/s]


In [18]:
# --- Visualization function for candidates, rays, and intersections using Open3D ---

def visualize_candidates_open3d(candidate_indices, ray_list=ray_list, candidate_list=candidate_list, intersection_objs=intersection_objs):
    """
    Visualize rays and intersections that construct the given candidates using Open3D.
    The visualization origin is shifted to the first ray origin used by the first candidate.
    Args:
        candidate_indices: list of candidate indices to visualize
        ray_list: list of Ray objects
        candidate_list: list of Candidate objects
        intersection_objs: list of Intersection objects
    """
    import open3d as o3d
    import numpy as np

    # Collect all rays and intersection points used by the selected candidates
    all_ray_indices = set()
    all_intersection_points = []
    candidate_centers = []
    for cid in candidate_indices:
        cand = candidate_list[cid]
        candidate_centers.append(np.array(cand.center))
        for iid in cand.intersections:
            inter = intersection_objs[iid]
            all_intersection_points.append(np.array(inter.point))
            all_ray_indices.update(inter.ray_pair)

    if not all_ray_indices:
        print("No rays found for the selected candidates.")
        return

    # Shift origin to the first ray's origin
    ray_indices_sorted = sorted(list(all_ray_indices))
    first_ray_origin = np.array(ray_list[ray_indices_sorted[0]].origin)
    def shift(pt):
        return np.array(pt) - first_ray_origin

    # Create Open3D geometries
    geometries = []

    # Rays as colored lines
    ray_colors = []
    ray_lines = []
    ray_points = []
    ray_length = 10.0  # meters, for visualization

    for idx, ridx in enumerate(ray_indices_sorted):
        ray = ray_list[ridx]
        origin = shift(ray.origin)
        direction = np.array(ray.direction)
        direction = direction / (np.linalg.norm(direction) + 1e-8)
        end = origin + direction * ray_length
        ray_points.append(origin)
        ray_points.append(end)
        ray_lines.append([2*idx, 2*idx+1])
        ray_colors.append([0.2, 0.2, 1.0])  # blue

    if ray_points:
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(np.array(ray_points))
        line_set.lines = o3d.utility.Vector2iVector(np.array(ray_lines))
        line_set.colors = o3d.utility.Vector3dVector(np.array(ray_colors * len(ray_lines)))
        geometries.append(line_set)

    # Intersection points as red spheres
    for pt in all_intersection_points:
        mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.15)
        mesh_sphere.translate(shift(pt))
        mesh_sphere.paint_uniform_color([1.0, 0.0, 0.0])
        geometries.append(mesh_sphere)

    # Candidate centers as green spheres
    for pt in candidate_centers:
        mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
        mesh_sphere.translate(shift(pt))
        mesh_sphere.paint_uniform_color([0.0, 1.0, 0.0])
        geometries.append(mesh_sphere)

    # Visualize
    o3d.visualization.draw_geometries(geometries)

In [20]:
visualize_candidates_open3d([390,393,394,395,399])

# Metrics

In [16]:
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

range_limit = 20
pred_path = '/mnt/Data/StreetView/scripts/triangulation_points_pred_20m.gpkg'
traject = pd.read_csv('/mnt/Data/StreetView/data/neuchatel/ne_traject.csv')
# Construct geometry from x_m_, y_m_, z_m_
traject['geometry'] = traject.apply(lambda row: Point(row['x_m_'], row['y_m_'], row['z_m_']), axis=1)

# Also create a 2D geometry column for XY plane
traject['geometry_xy'] = traject.apply(lambda row: Point(row['x_m_'], row['y_m_']), axis=1)

gt_path = '/mnt/Data/StreetView/data/neuchatel/NE_GT_3D.gpkg'
gt_gdf = gpd.read_file(gt_path, layer='ne_gt_3d')

# --- Matrix calculation for filtering gt_gdf by nearby traject points in XY ---

# Get centroid XY coordinates for all GT geometries
gt_centroids = gt_gdf.geometry.centroid
gt_centroids_xy = np.array([[pt.x, pt.y] for pt in gt_centroids])

# Get all traject XY coordinates
traject_xy = np.array([[pt.x, pt.y] for pt in traject['geometry_xy']])

# Compute distance matrix: shape (n_gt, n_traject)
dists_matrix = np.linalg.norm(gt_centroids_xy[:, None, :] - traject_xy[None, :, :], axis=2)

# Count how many traject points are within 15 meters for each GT centroid
nearby_counts = (dists_matrix <= range_limit).sum(axis=1)

# Filter gt_gdf: keep only rows where at least 3 traject points are within 15m
gt_gdf = gt_gdf[nearby_counts >= 3].reset_index(drop=True)


pred_gdf = gpd.read_file(pred_path)
pred_gdf = pred_gdf[(pred_gdf.mean_intersection_length <= range_limit).values & (pred_gdf.mean_intersection_dist <= 0.1).values]

# --- Matrix calculation for mean nearest traject distance in 3D ---

# Prepare pred_gdf XYZ coordinates
pred_xyz = np.stack([pred_gdf.geometry.x.values, pred_gdf.geometry.y.values, pred_gdf.elevation.values], axis=1)
# Prepare traject XYZ coordinates
traject_xyz = np.array([[pt.x, pt.y, pt.z] for pt in traject['geometry']])

# Compute distance matrix: shape (n_pred, n_traject)
dists_3d_matrix = np.linalg.norm(pred_xyz[:, None, :] - traject_xyz[None, :, :], axis=2)

# For each pred point, get mean of nearest 5 traject distances
k = 10
partitioned = np.partition(dists_3d_matrix, k-1, axis=1)[:, :k]
mean_nearest_traject_dist = partitioned.mean(axis=1)

pred_gdf = pred_gdf.assign(mean_nearest_traject_dist=mean_nearest_traject_dist)
pred_gdf = pred_gdf[pred_gdf['mean_intersection_length'] <= 5 + pred_gdf['mean_nearest_traject_dist']].copy()

# pred_gdf.to_file(f"triangulation_points_pred_demo.gpkg", driver="GPKG")

In [13]:
# Project GT geometries to XY (2D) polygons
gt_polys_2d = gt_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Polygon' else geom.convex_hull)
# Project pred points to XY (2D) points
pred_points_2d = pred_gdf['geometry'].apply(lambda pt: Point(pt.x, pt.y))

# For each GT polygon, find the closest pred point (in XY), and check if it is inside the polygon
gt_matched_pred_idx = {}  # gt_idx -> pred_idx if matched
pred_matched_gt_idx = {}  # pred_idx -> gt_idx if matched

distances = []

for gt_idx, gt_poly in gt_polys_2d.items():
    # Find closest pred point to this GT polygon (by centroid XY)
    gt_centroid = gt_poly.centroid
    pred_xy = np.array([[pt.x, pt.y] for pt in pred_points_2d])
    gt_xy = np.array([gt_centroid.x, gt_centroid.y])
    dists = np.linalg.norm(pred_xy - gt_xy, axis=1)
    closest_pred_idx = dists.argmin()
    closest_pred_point = pred_points_2d.iloc[closest_pred_idx]
    # Check if closest pred point is inside the GT polygon
    if gt_poly.contains(closest_pred_point):
        gt_matched_pred_idx[gt_idx] = closest_pred_idx
        pred_matched_gt_idx[closest_pred_idx] = gt_idx
        # Save XY distance for stats
        distances.append(np.linalg.norm([closest_pred_point.x - gt_centroid.x, closest_pred_point.y - gt_centroid.y]))

# True Positives: number of matched GT polygons (one-to-one)
TP = len(gt_matched_pred_idx)
# False Positives: pred points not matched to any GT polygon
FP = len(pred_points_2d) - len(pred_matched_gt_idx)
# False Negatives: GT polygons not matched to any pred point
FN = len(gt_polys_2d) - len(gt_matched_pred_idx)

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Distance statistics
if distances:
    distances_np = np.array(distances)
    dist_stats = {
        'mean': np.mean(distances_np),
        'std': np.std(distances_np),
        'min': np.min(distances_np),
        'max': np.max(distances_np),
        'median': np.median(distances_np),
        'count': len(distances_np)
    }
else:
    dist_stats = {}

print("True Positives:", TP)
print("False Positives:", FP)
print("False Negatives:", FN)
print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("f1_score: {:.3f}".format(f1_score))
print("Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:")
for k, v in dist_stats.items():
    print(f"  {k}: {v}")


True Positives: 1185
False Positives: 162
False Negatives: 203
Precision: 0.880
Recall: 0.854
f1_score: 0.867
Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:
  mean: 0.061965935666899535
  std: 0.043294714196046004
  min: 0.001675845673727821
  max: 0.3331082248863427
  median: 0.05323019294645722
  count: 1185


In [15]:
# Project GT geometries to XY (2D) polygons
gt_polys_2d = gt_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Polygon' else geom.convex_hull)
# Project pred points to XY (2D) points
pred_points_2d = pred_gdf['geometry'].apply(lambda pt: Point(pt.x, pt.y))

# For each GT polygon, find the closest pred point (in XY), and check if it is inside the polygon
gt_matched_pred_idx = {}  # gt_idx -> pred_idx if matched
pred_matched_gt_idx = {}  # pred_idx -> gt_idx if matched

distances = []

for gt_idx, gt_poly in gt_polys_2d.items():
    # Find closest pred point to this GT polygon (by centroid XY)
    gt_centroid = gt_poly.centroid
    pred_xy = np.array([[pt.x, pt.y] for pt in pred_points_2d])
    gt_xy = np.array([gt_centroid.x, gt_centroid.y])
    dists = np.linalg.norm(pred_xy - gt_xy, axis=1)
    closest_pred_idx = dists.argmin()
    closest_pred_point = pred_points_2d.iloc[closest_pred_idx]
    # Check if closest pred point is inside the GT polygon
    if gt_poly.contains(closest_pred_point):
        gt_matched_pred_idx[gt_idx] = closest_pred_idx
        pred_matched_gt_idx[closest_pred_idx] = gt_idx
        # Save XY distance for stats
        distances.append(np.linalg.norm([closest_pred_point.x - gt_centroid.x, closest_pred_point.y - gt_centroid.y]))

# True Positives: number of matched GT polygons (one-to-one)
TP = len(gt_matched_pred_idx)
# False Positives: pred points not matched to any GT polygon
FP = len(pred_points_2d) - len(pred_matched_gt_idx)
# False Negatives: GT polygons not matched to any pred point
FN = len(gt_polys_2d) - len(gt_matched_pred_idx)

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Distance statistics
if distances:
    distances_np = np.array(distances)
    dist_stats = {
        'mean': np.mean(distances_np),
        'std': np.std(distances_np),
        'min': np.min(distances_np),
        'max': np.max(distances_np),
        'median': np.median(distances_np),
        'count': len(distances_np)
    }
else:
    dist_stats = {}

print("True Positives:", TP)
print("False Positives:", FP)
print("False Negatives:", FN)
print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("f1_score: {:.3f}".format(f1_score))
print("Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:")
for k, v in dist_stats.items():
    print(f"  {k}: {v}")


True Positives: 1131
False Positives: 133
False Negatives: 257
Precision: 0.895
Recall: 0.815
f1_score: 0.853
Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:
  mean: 0.060989909553414094
  std: 0.042109058388992045
  min: 0.001675845673727821
  max: 0.2968191938137795
  median: 0.05305985676038003
  count: 1131


In [17]:
# Project GT geometries to XY (2D) polygons
gt_polys_2d = gt_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Polygon' else geom.convex_hull)
# Project pred points to XY (2D) points
pred_points_2d = pred_gdf['geometry'].apply(lambda pt: Point(pt.x, pt.y))

# For each GT polygon, find the closest pred point (in XY), and check if it is inside the polygon
gt_matched_pred_idx = {}  # gt_idx -> pred_idx if matched
pred_matched_gt_idx = {}  # pred_idx -> gt_idx if matched

distances = []

for gt_idx, gt_poly in gt_polys_2d.items():
    # Find closest pred point to this GT polygon (by centroid XY)
    gt_centroid = gt_poly.centroid
    pred_xy = np.array([[pt.x, pt.y] for pt in pred_points_2d])
    gt_xy = np.array([gt_centroid.x, gt_centroid.y])
    dists = np.linalg.norm(pred_xy - gt_xy, axis=1)
    closest_pred_idx = dists.argmin()
    closest_pred_point = pred_points_2d.iloc[closest_pred_idx]
    # Check if closest pred point is inside the GT polygon
    if gt_poly.contains(closest_pred_point):
        gt_matched_pred_idx[gt_idx] = closest_pred_idx
        pred_matched_gt_idx[closest_pred_idx] = gt_idx
        # Save XY distance for stats
        distances.append(np.linalg.norm([closest_pred_point.x - gt_centroid.x, closest_pred_point.y - gt_centroid.y]))

# True Positives: number of matched GT polygons (one-to-one)
TP = len(gt_matched_pred_idx)
# False Positives: pred points not matched to any GT polygon
FP = len(pred_points_2d) - len(pred_matched_gt_idx)
# False Negatives: GT polygons not matched to any pred point
FN = len(gt_polys_2d) - len(gt_matched_pred_idx)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Distance statistics
if distances:
    distances_np = np.array(distances)
    dist_stats = {
        'mean': np.mean(distances_np),
        'std': np.std(distances_np),
        'min': np.min(distances_np),
        'max': np.max(distances_np),
        'median': np.median(distances_np),
        'count': len(distances_np)
    }
else:
    dist_stats = {}

print("True Positives:", TP)
print("False Positives:", FP)
print("False Negatives:", FN)
print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("f1_score: {:.3f}".format(f1_score))
print("Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:")
for k, v in dist_stats.items():
    print(f"  {k}: {v}")


True Positives: 1204
False Positives: 197
False Negatives: 184
Precision: 0.859
Recall: 0.867
f1_score: 0.863
Distance statistics (meters) between predicted point and GT polygon centroid in XY plane:
  mean: 0.062448262938878875
  std: 0.043845324651224746
  min: 0.001675845673727821
  max: 0.3331082248863427
  median: 0.053422706076573165
  count: 1204
